In [1]:
import datetime
import os

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [2]:
# Define the project's root directory (replace with your actual path)
project_root = "/Users/sunnythesage/PythonProjects/Data-Science-BootCamp/03-Deep-Learning-BootCamp/7 - End to End Deep Learning Project Using ANN/advanced-customer-churn-analysis-using-ann"

# Change the current working directory to the project's root
os.chdir(project_root)

# Define the artifacts directory path
artifacts_dir = os.path.join(os.getcwd(), 'artifacts')

# Create the artifacts directory if it doesn't exist
os.makedirs(artifacts_dir, exist_ok = True)

In [3]:
## Load the dataset
data = pd.read_csv('data/raw/churn-modelling-dataset.csv')

data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Preprocess the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)

In [5]:
# Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [6]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [7]:
# One-hot encode 'Geography'
onehot_encoder_geo = OneHotEncoder(handle_unknown = 'ignore')

geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = onehot_encoder_geo.get_feature_names_out(['Geography']))

geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
# Combine one-hot encoded column with original data
data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df], axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
# Split the data into features and target
X = data.drop('EstimatedSalary', axis = 1)
y = data['EstimatedSalary']

In [10]:
## Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [11]:
## Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### ANN Regression Problem statement

In [12]:
import keras
from keras import layers

In [13]:
# Build Our ANN Model
model = keras.Sequential([
    layers.Input(shape = (X_train.shape[1],)),  # Explicitly define the input shape
    layers.Dense(64, activation = 'relu'),  # HL1
    layers.Dense(32, activation = 'relu'),  # HL2
    layers.Dense(1, activation = 'sigmoid')  # Output layer
])

# Compile the model
opt = keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Define the root log directory
root_log_dir = os.path.join(os.getcwd(), 'logs')
os.makedirs(root_log_dir, exist_ok = True)

# Generate a unique log directory for each run
log_dir = os.path.join(root_log_dir, 'fit', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
os.makedirs(log_dir, exist_ok = True)

In [15]:
# Set up TensorBoard
log_dir = "regression_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Create the TensorBoard callback
tensorboard_callback = keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [16]:
# Set up Early Stopping
early_stopping_callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10,
                                                        restore_best_weights = True)

In [17]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs = 100,
    callbacks = [tensorboard_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: -903320512.0000 - val_accuracy: 0.0000e+00 - val_loss: -17864368128.0000
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: -43371737088.0000 - val_accuracy: 0.0000e+00 - val_loss: -172152340480.0000
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step - accuracy: 0.0000e+00 - loss: -260508745728.0000 - val_accuracy: 0.0000e+00 - val_loss: -597679865856.0000
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step - accuracy: 0.0000e+00 - loss: -783707471872.0000 - val_accuracy: 0.0000e+00 - val_loss: -1391625371648.0000
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - accuracy: 0.0000e+00 - loss: -1680681861120.0000 - val_accuracy: 0.0000e+00 - val_loss: -2627335094272.0000
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: -3125090975744.0000 - val_accuracy: 0.0000e+00 - val_loss: -4360640135168.0000
Epoch 7/100
250/250 ━━━━━━━━

In [18]:
## Evaluate model on the test data
test_loss, test_mae = model.evaluate(X_test, y_test)

print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step - accuracy: 0.0000e+00 - loss: -5541617524015104.0000
Test MAE : 0.0


In [19]:
# Save the model in the artifacts directory with .keras extension
keras.saving.save_model(model, os.path.join(artifacts_dir, 'regression_model.keras'))

In [20]:
# Load Tensorboard Extension
%load_ext tensorboard

%tensorboard --logdir logs/fit